In [16]:
import os 
import pandas as pd 
from bs4 import BeautifulSoup
from io import StringIO

In [ ]:
%pip install pandas

In [ ]:
SCORE_DIR = "data/scores"

In [ ]:
box_scores = os.listdir(SCORE_DIR)

In [ ]:
box_scores

In [ ]:
len(box_scores)

In [ ]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

In [ ]:
box_scores

In [186]:

def parse_html(box_score):
    with open(box_score, encoding="utf-8") as f:  
        html = f.read()
    soup = BeautifulSoup(html, "html.parser")  
    [s.decompose() for s in soup.select("tr.over_header")] 
    return soup
    
    
    

In [187]:
import pandas as pd
from io import StringIO

def read_line_score(soup):
    """Extract the line score table, handling missing or malformed HTML gracefully."""
    try:
        html_string = str(soup)
        html_buffer = StringIO(html_string)
        tables = pd.read_html(html_buffer, attrs={"id": "line_score"})

        if not tables:
            print("Warning: 'line_score' table not found.")
            return None  
        
        line_score = tables[0]

        cols = list(line_score.columns)
        cols[0] = "team"
        cols[-1] = "total"
        line_score.columns = cols

        line_score = line_score[["team", "total"]]

        return line_score
    
    except Exception as e:
        print(f"Error reading line score: {e}")
        return None  


In [188]:

def read_stats(soup, team, stat):
    """Reads a stats table for a given team and stat type."""
    html_string = str(soup)
    html_buffer = StringIO(html_string) 
    df = pd.read_html(html_buffer, attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df  

In [189]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [190]:
base_cols = None
games = []
box_score = box_scores[0]

for idx, box_score in enumerate(box_scores):
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    if line_score is None:
        print(f"Skipping game {idx} (issue with line score)")
        continue

    teams = list(line_score["team"])
    summaries = []
    
    for team in teams:
        try:
            basic = read_stats(soup, team, "basic")
            advanced = read_stats(soup, team, "advanced")

            totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
            totals.index = totals.index.str.lower()

            maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
            maxes.index = maxes.index.str.lower() + "_max"

            summary = pd.concat([totals, maxes])

            if base_cols is None:
                base_cols = list(summary.index.drop_duplicates(keep="first"))
                base_cols = [b for b in base_cols if "bpm" not in b]

            summary = summary[base_cols]
            summaries.append(summary)
        
        except Exception as e:
            print(f"Skipping team {team} in game {idx} due to error: {e}")
            continue
    if not summaries:
        print(f"Skipping game {idx} (no valid summaries)")
        continue

    summary = pd.concat(summaries, axis=1).T
    game = pd.concat([summary, line_score], axis=1)
    game["home"] = [0,1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["seasons"] = read_season_info(soup)
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    full_game["won"] = full_game["total"] > full_game["total_opp"]

    games.append(full_game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)} processed")


100 / 9769 processed
200 / 9769 processed
300 / 9769 processed
400 / 9769 processed
500 / 9769 processed
600 / 9769 processed
700 / 9769 processed
800 / 9769 processed
900 / 9769 processed
1000 / 9769 processed
1100 / 9769 processed
Error reading line score: no text parsed from document (line 0)
Skipping game 1100 (issue with line score)
Error reading line score: no text parsed from document (line 0)
Skipping game 1144 (issue with line score)
Error reading line score: no text parsed from document (line 0)
Skipping game 1145 (issue with line score)
Error reading line score: no text parsed from document (line 0)
Skipping game 1149 (issue with line score)
1200 / 9769 processed
1300 / 9769 processed
1400 / 9769 processed
1500 / 9769 processed
1600 / 9769 processed
1700 / 9769 processed
1800 / 9769 processed
1900 / 9769 processed
2000 / 9769 processed
2100 / 9769 processed
2200 / 9769 processed
2300 / 9769 processed
2400 / 9769 processed
2500 / 9769 processed
2600 / 9769 processed
2700 / 97

In [191]:
#soup

In [192]:
#teams

In [193]:
#df

In [194]:
#advanced

In [195]:
#basic

In [196]:
#total

In [197]:
#maxes

In [198]:
#totals

In [199]:
#summary

In [200]:
#game

In [201]:
#game_opp

In [202]:
#full_game

In [203]:
#box_score

In [204]:
games_df = pd.concat(games, ignore_index=True)

In [205]:
games_df

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,seasons,date,won
0,240.0,240.0,36.0,88.0,0.409,8.0,32.0,0.250,19.0,25.0,...,31.6,27.3,138.0,107.0,CLE,102,1,2018,2017-10-17,False
1,240.0,240.0,38.0,83.0,0.458,5.0,22.0,0.227,21.0,25.0,...,34.7,29.9,129.0,112.0,BOS,99,0,2018,2017-10-17,True
2,240.0,240.0,47.0,97.0,0.485,15.0,41.0,0.366,13.0,19.0,...,31.7,32.6,250.0,127.0,GSW,121,1,2018,2017-10-17,True
3,240.0,240.0,43.0,80.0,0.538,16.0,30.0,0.533,19.0,21.0,...,18.2,31.2,152.0,126.0,HOU,122,0,2018,2017-10-17,False
4,240.0,240.0,38.0,76.0,0.500,7.0,21.0,0.333,25.0,30.0,...,33.3,43.9,147.0,119.0,BOS,100,1,2018,2017-10-18,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19525,240.0,240.0,40.0,77.0,0.519,7.0,19.0,0.368,23.0,29.0,...,25.8,35.8,228.0,119.0,PHI,105,0,2025,2025-02-26,True
19526,240.0,240.0,46.0,87.0,0.529,15.0,36.0,0.417,11.0,12.0,...,20.0,25.7,203.0,129.0,UTA,101,1,2025,2025-02-26,True
19527,240.0,240.0,38.0,84.0,0.452,16.0,44.0,0.364,9.0,17.0,...,25.0,28.2,174.0,111.0,SAC,118,0,2025,2025-02-26,False
19528,240.0,240.0,45.0,101.0,0.446,12.0,37.0,0.324,27.0,35.0,...,75.0,28.6,194.0,128.0,WAS,121,1,2025,2025-02-26,True


In [206]:
games_df.to_csv("nba_games.csv")